# Retrieval Augmented Generation with MongoDB and Gemma
* Notebook by Adam Lang
* Date: 2/25/2025

# Overview
* In this notebook we will build a RAG system using MongoDB vector database using open source models from Google Gemma and hugging face.

# Dataset
* We will leverage an open source dataset from hugging face.
* Dataset name: `MongoDB/embedded_movies`
  * dataset_card: https://huggingface.co/datasets/MongoDB/embedded_movies


# Gemma LLM
* We will use the open source Gemma model from hugging face
* model: `google/gemma-2b-it`
* model card: https://huggingface.co/google/gemma-2b-it

# Vector DB
* As I mentioned we will test out MongoDB Atlas and its various vector search capabilities. 

# Install Dependencies

In [7]:
%%capture
!pip install pandas sentence_transformers

Note:
* The pymongo version you install depends on if you are using:
  1. Driver
  2. Compass GUI
  3. Shell
  4. VS Code
* The version below is for the Driver.

In [8]:
%%capture
!pip install --upgrade pymongo

In [9]:
%%capture
!pip install transformers datasets

In [10]:
%%capture
!pip install --upgrade accelerate

In [11]:
## check version of accelerate
import accelerate
print(f"Accelerate version: {accelerate.__version__}")

Accelerate version: 1.4.0


# Hugging Face Notebook Login

In [12]:
## hf hub login
from huggingface_hub import notebook_login
notebook_login()

# Load Dataset and EDA
* We will start by loading a dataset from hugging face that is open source

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## ML imports
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

In [14]:
## now load the dataset
data = load_dataset('MongoDB/embedded_movies')

## view data
data

DatasetDict({
    train: Dataset({
        features: ['plot', 'runtime', 'genres', 'fullplot', 'directors', 'writers', 'countries', 'poster', 'languages', 'cast', 'title', 'num_mflix_comments', 'rated', 'imdb', 'awards', 'type', 'metacritic', 'plot_embedding'],
        num_rows: 1500
    })
})

In [15]:
## show column names
print(data['train'].column_names)

['plot', 'runtime', 'genres', 'fullplot', 'directors', 'writers', 'countries', 'poster', 'languages', 'cast', 'title', 'num_mflix_comments', 'rated', 'imdb', 'awards', 'type', 'metacritic', 'plot_embedding']


In [16]:
## lets convert this dict object to a pandas df
df = pd.DataFrame(data['train'])
df.head()

,plot,runtime,genres,fullplot,directors,writers,countries,poster,languages,cast,title,num_mflix_comments,rated,imdb,awards,type,metacritic,plot_embedding
0,Young Pauline is left a lot of money when her ...,199.0,[Action],Young Pauline is left a lot of money when her ...,"[Louis J. Gasnier, Donald MacKenzie]","[Charles W. Goddard (screenplay), Basil Dickey...",[USA],https://m.media-amazon.com/images/M/MV5BMzgxOD...,[English],"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",The Perils of Pauline,0,None,"{'id': 4465, 'rating': 7.6, 'votes': 744}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[0.0007293965299999999, -0.026834568000000003,..."
1,A penniless young man tries to save an heiress...,22.0,"[Comedy, Short, Action]",As a penniless man worries about how he will m...,"[Alfred J. Goulding, Hal Roach]",[H.M. Walker (titles)],[USA],https://m.media-amazon.com/images/M/MV5BNzE1OW...,[English],"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",From Hand to Mouth,0,TV-G,"{'id': 10146, 'rating': 7.0, 'votes': 639}","{'nominations': 1, 'text': '1 nomination.', 'w...",movie,NaN,"[-0.022837115, -0.022941574000000003, 0.014937..."
2,"Michael ""Beau"" Geste leaves England in disgrac...",101.0,"[Action, Adventure, Drama]","Michael ""Beau"" Geste leaves England in disgrac...",[Herbert Brenon],"[Herbert Brenon (adaptation), John Russell (ad...",[USA],None,[English],"[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",Beau Geste,0,None,"{'id': 16634, 'rating': 6.9, 'votes': 222}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[0.00023330492999999998, -0.028511643000000003..."
3,"Seeking revenge, an athletic young man joins t...",88.0,"[Adventure, Action]",A nobleman vows to avenge the death of his fat...,[Albert Parker],"[Douglas Fairbanks (story), Jack Cunningham (a...",[USA],https://m.media-amazon.com/images/M/MV5BMzU0ND...,None,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",The Black Pirate,1,None,"{'id': 16654, 'rating': 7.2, 'votes': 1146}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[-0.005927917, -0.033394486, 0.0015323418, -0...."
4,An irresponsible young millionaire changes his...,58.0,"[Action, Comedy, Romance]","The Uptown Boy, J. Harold Manners (Lloyd) is a...",[Sam Taylor],"[Ted Wilde (story), John Grey (story), Clyde B...",[USA],https://m.media-amazon.com/images/M/MV5BMTcxMT...,[English],"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",For Heaven's Sake,0,PASSED,"{'id': 16895, 'rating': 7.6, 'votes': 918}","{'nominations': 1, 'text': '1 nomination.', 'w...",movie,NaN,"[-0.0059373598, -0.026604708, -0.0070914757000..."


In [17]:
## info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   plot                1473 non-null   object 
 1   runtime             1485 non-null   float64
 2   genres              1500 non-null   object 
 3   fullplot            1452 non-null   object 
 4   directors           1487 non-null   object 
 5   writers             1487 non-null   object 
 6   countries           1500 non-null   object 
 7   poster              1411 non-null   object 
 8   languages           1499 non-null   object 
 9   cast                1499 non-null   object 
 10  title               1500 non-null   object 
 11  num_mflix_comments  1500 non-null   int64  
 12  rated               1192 non-null   object 
 13  imdb                1500 non-null   object 
 14  awards              1500 non-null   object 
 15  type                1500 non-null   object 
 16  metacr

In [18]:
## any nulls?
df.isnull().sum().sort_values(ascending=False)

metacritic            928
rated                 308
poster                 89
fullplot               48
plot_embedding         28
plot                   27
runtime                15
writers                13
directors              13
languages               1
cast                    1
countries               0
title                   0
num_mflix_comments      0
imdb                    0
awards                  0
type                    0
genres                  0
dtype: int64

Summary
* There are null values so we will have to handle those when building a RAG system.

In [19]:
## drop null values in the `fullplot` column --> then create new embeddings
df = df.dropna(subset=['fullplot'])
print("\nNumber of missing values in each column after dropping fullplot nulls: \n\n")
print(df.isnull().sum().sort_values(ascending=False))


Number of missing values in each column after dropping fullplot nulls: 


metacritic            893
rated                 279
poster                 78
runtime                14
writers                13
directors              12
cast                    1
languages               1
plot_embedding          1
countries               0
title                   0
num_mflix_comments      0
fullplot                0
imdb                    0
awards                  0
type                    0
genres                  0
plot                    0
dtype: int64


In [20]:
## now drop the plot_embeddings col to create new embeddings
df = df.drop(columns=['plot_embedding'])
df.columns

Index(['plot', 'runtime', 'genres', 'fullplot', 'directors', 'writers',
       'countries', 'poster', 'languages', 'cast', 'title',
       'num_mflix_comments', 'rated', 'imdb', 'awards', 'type', 'metacritic'],
      dtype='object')

Summary
* Great we succesfully dropped the `plot_embedding` column.

# Create Embeddings
* We can now create embeddings of the `fullplot` column instead.
* Embedding model: `thenlper/gte-large`
  * embed model card: https://huggingface.co/thenlper/gte-large
* These embeddings have a dimension of 1024 and sequence length of 512. On most MTEB tasks these perform better than the standard SentenceTransformer models.
* Original paper: https://arxiv.org/abs/2308.03281

In [21]:
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
from sentence_transformers import SentenceTransformer

tqdm_notebook().pandas()

# Load embed model
embed_model = SentenceTransformer("thenlper/gte-large")

def create_embedding(text: str) -> list[float]:
    """Function that takes in a string and returns embeddings as a list of floats"""
    if not text.strip():
        print("Tried to create embedding on empty text.")
        return []
    embeddings = embed_model.encode(text)
    return embeddings.tolist()

# Apply function to dataset
df['embedding'] = df['fullplot'].progress_apply(create_embedding)  # Note: Changed to 'embedding' to match the index


0it [00:00, ?it/s]

  0%|          | 0/1452 [00:00<?, ?it/s]

In [22]:
## view index of embeddings
df['embedding'][0:2]

0    [-0.00928583275526762, -0.005062101874500513, ...
1    [-0.00243937224149704, 0.02309592068195343, -0...
Name: embedding, dtype: object

# MongoDB Vector DB setup
* For this section you need to create a free MongoDB Atlas cluster.
* Steps to setup the Vector Search Index:
  * 1. Go to MongoDB Atlas --> create free account
  * 2. Click on "create cluster" (choose free tier)
  * 3. Then create a "new database" or collection.
    * Give it a name such as "Movie_RAG" as we are working with movie data.
  * 4. Go to "Search Index" -- this is where you create a vector index.
  * 5. Go to "Create Search Index".
  * 6. Create Configuration
    * Choose **Vector Search** which is semantic search for AI applications.
    * Give name to the index such as `vector_index`
    * Assign the vector_index to the collection (database) you created.
    * Choose Configuration Method: Atlas Vector Search Index with a **JSON Editor**
    * Edit the JSON file like this below. The `numDimensions` corresponds to your specific embeddings that you are using.
      * Note: Embedding dimensions obviously depend on the dimensionality of your data, but also on your individual storage capacity in your database so make sure this is correct before proceeding with a specific embedding model.
```
{
  "fields": [
    {
      "type": "vector",
      "path": "embedding",
      "numDimensions": 1024,
      "similarity": "cosine"
    }
  ]
}
```
* We have to initiate the MongoDB client.
* Then we can index and ingest the embeddings we just created into the vector DB.

## MongoDB IP Address Access
* In order to connect to your MongoDB instance you need to make sure your IP address is permitted in the `Network Access` of the database.
* Here are the steps (assuming you already have a cluster and collection created):

1. In the MongoDB Atlas dashboard, go to the “Network Access” tab from the left sidebar.
2. Click on the “Add IP Address” button.
3. Add your current IP address to the whitelist to allow connections from your location or you can allow "access from anywhere".

In [23]:
%%capture
!pip install certifi==2023.7.22 ## collection of up-to-date root certificates used during SSL handshake

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
# ## alternative way to connect
# import pymongo
# from pymongo import MongoClient

# ## init mongo client
# client = pymongo.MongoClient("mongodb+srv://<username>:<password>@<cluster-address>/<dbname>?retryWrites=true&w=majority")

# ## access db
# db = client.<dbname>
# collection = db.<collectionname>

In [25]:
import os 
from getpass import getpass

MONGO_URI = getpass("Enter your MONGO URI: ")

Enter your MONGO URI:  ········


In [26]:
## set env vars
os.environ['MONGO_URI'] = MONGO_URI

In [27]:
import pymongo
import certifi



## establish connection to mongo DB
def get_mongo_client(mongo_uri):
    """Function to establish connection to Mongo DB"""
    try:
        #client = pymongo.MongoClient(mongo_uri, ssl=True, ssl_cert_reqs=ssl.CERT_NONE) ## disables SSL -- not good for prod but good for testing
        client = pymongo.MongoClient(mongo_uri, tls=True, tlsAllowInvalidCertificates=True)
        print("Connection to MongoDB successful!")
        return client 
    except pymongo.errors.ConnectionFailure as e: 
        print(f"Connection failed: {e}")

mongo_uri = MONGO_URI
if not mongo_uri:
    print(f"MONGO_URI not set in environment variables")

## init mongo client
mongo_client = get_mongo_client(mongo_uri)

# Access database and collection
db = mongo_client["movies_rag"]
collection = db["movies_collection_1"]

Connection to MongoDB successful!


## Test Connection to DB

In [28]:
# List databases
print(mongo_client.list_database_names())

# List collections in your database
print(db.list_collection_names())

['movies_rag', 'sample_airbnb', 'sample_analytics', 'sample_geospatial', 'sample_guides', 'sample_mflix', 'sample_restaurants', 'sample_supplies', 'sample_training', 'sample_weatherdata', 'admin', 'local']
['movies_collection_1']


## Clean out any existing collections

In [29]:
## Clean out any existing collections
try:
    delete_result = collection.delete_many({})
    if delete_result.deleted_count > 0:
        print(f"Deleted {delete_result.deleted_count} documents.")
    else:
        print("No documents to delete. Collection is already empty.")
except pymongo.errors.ServerSelectionTimeoutError as e:
    print(f"Error connecting to MongoDB: {e}")
    print(
        "Possible causes: Network issues, incorrect MongoDB URI, or MongoDB server problems."
    )
    print("Please check your connection string and network access in Atlas.")

Deleted 1452 documents.


# Data Ingestion into MongoDB

In [30]:
## ingest documents
## 1. convert to dict
documents = df.to_dict('records')

## 2. insert docs
collection.insert_many(documents)

print("Data ingestion into MongoDB completed!")


Data ingestion into MongoDB completed!


# Vector Index Creation
* This step can be completed "manually" in the user interface of MongoDB, but it is easily done this way programmatically.
* First we check the vector index if it exists, if it already does we wont create a new one, if it doesnt we will.

In [31]:
from pymongo.operations import SearchIndexModel

## check first to see if vector index exists ---> if not create it
def check_and_create_vector_index(collection, index_name="vector_index"):
    # List existing search indexes
    existing_indexes = collection.list_search_indexes()
    
    # Check if the vector index already exists
    index_exists = any(index['name'] == index_name for index in existing_indexes)
    
    if index_exists:
        print(f"Vector index '{index_name}' already exists.")
        return

    # If the index doesn't exist, create it
    index_definition = {
        "mappings": {
            "dynamic": True,
            "fields": {
                "embedding": {
                    "dimensions": 1024,  # Adjust this to match your actual embedding size
                    "similarity": "cosine",
                    "type": "knnVector"
                }
            }
        }
    }
    
    try:
        index_model = SearchIndexModel(index_definition, name=index_name)
        collection.create_search_index(index_model)
        print(f"Vector index '{index_name}' created successfully.")
    except Exception as e:
        print(f"Error creating vector index: {e}")

# Usage
check_and_create_vector_index(collection)

Vector index 'vector_index' already exists.


# Check embedding dimensions for each document

In [32]:
## Verify that the embeddings are correctly stored:
def check_embeddings(collection, num_docs=5):
    for doc in collection.find().limit(num_docs):
        print(f"Document ID: {doc['_id']}")
        print(f"Title: {doc.get('title', 'N/A')}")
        embedding = doc.get('embedding')
        if embedding:
            print(f"Embedding length: {len(embedding)}")
        else:
            print("Embedding not found")
        print("---")

check_embeddings(collection)

Document ID: 67be09b7e3fd5d8914d8b14e
Title: The Perils of Pauline
Embedding length: 1024
---
Document ID: 67be09b7e3fd5d8914d8b14f
Title: From Hand to Mouth
Embedding length: 1024
---
Document ID: 67be09b7e3fd5d8914d8b150
Title: Beau Geste
Embedding length: 1024
---
Document ID: 67be09b7e3fd5d8914d8b151
Title: The Black Pirate
Embedding length: 1024
---
Document ID: 67be09b7e3fd5d8914d8b152
Title: For Heaven's Sake
Embedding length: 1024
---


# Setup Semantic Search functions
* We need a few functions before we can implement this.

In [33]:
## lets Verify the index exists: Make sure the "vector_index" exists in the collection:

print(f"Indexes in collection: {collection.index_information()}")

Indexes in collection: {'_id_': {'v': 2, 'key': [('_id', 1)]}}


In [34]:
#Check if the collection is empty: Add a check to see if there are any documents in the collection:

print(f"Number of documents in collection: {collection.count_documents({})}")

Number of documents in collection: 1452


In [35]:
def check_embeddings(collection, num_docs=5):
    for doc in collection.find().limit(num_docs):
        print(f"Document ID: {doc['_id']}")
        print(f"Title: {doc.get('title', 'N/A')}")
        print(f"Embeddings: {doc.get('embedding', 'Not found')}")
        print("---")

check_embeddings(collection)

Document ID: 67be09b7e3fd5d8914d8b14e
Title: The Perils of Pauline
Embeddings: [-0.00928583275526762, -0.005062101874500513, -0.010958139784634113, 0.029197748750448227, -0.0032040588557720184, 0.006529400125145912, -0.0005196502897888422, 0.034043777734041214, 0.004956729710102081, -0.005387570708990097, 0.02813304215669632, 0.005806769710034132, 0.008509224280714989, -0.006339729763567448, -0.026832520961761475, 0.0007219529361464083, -0.052018679678440094, -0.018970800563693047, -0.03467943146824837, -0.014942395500838757, 0.021869568154215813, 0.013537668623030186, -0.07247655838727951, -0.03980889171361923, -0.005702628754079342, 0.037348125129938126, 0.03697190061211586, -0.000929308938793838, 0.05613444373011589, 0.04322363808751106, -0.016124606132507324, -0.0181397944688797, 0.018382973968982697, -0.031032327562570572, -0.006974264979362488, -0.019166946411132812, 0.04368210956454277, -0.0282314233481884, -0.0007613821653649211, -0.06701456755399704, 0.014413019642233849, -0.0

# Lets see all the database indices before we proceed

In [36]:
def list_all_indexes(collection):
    print("Standard indexes:")
    for index in collection.list_indexes():
        print(index)

    print("\nSearch indexes:")
    try:
        search_indexes = collection.list_search_indexes()
        for index in search_indexes:
            print(index)
    except Exception as e:
        print(f"Error listing search indexes: {e}")

list_all_indexes(collection)

Standard indexes:
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])

Search indexes:
{'id': '67bdd710e5f29f437f590cfb', 'name': 'vector_index', 'type': 'vectorSearch', 'status': 'READY', 'queryable': True, 'latestDefinitionVersion': {'version': 0, 'createdAt': datetime.datetime(2025, 2, 25, 14, 43, 29, 361000)}, 'latestDefinition': {'fields': [{'type': 'vector', 'path': 'embedding', 'numDimensions': 1024, 'similarity': 'cosine'}]}, 'statusDetail': [{'hostname': 'atlas-omze7r-shard-00-00', 'status': 'READY', 'queryable': True, 'mainIndex': {'status': 'READY', 'queryable': True, 'definitionVersion': {'version': 0, 'createdAt': datetime.datetime(2025, 2, 25, 14, 43, 29)}, 'definition': {'fields': [{'type': 'vector', 'path': 'embedding', 'numDimensions': 1024, 'similarity': 'cosine'}]}}}, {'hostname': 'atlas-omze7r-shard-00-01', 'status': 'READY', 'queryable': True, 'mainIndex': {'status': 'READY', 'queryable': True, 'definitionVersion': {'version': 0, 'createdAt': datetime.dat

In [37]:
# print a sample document to verify fields
def print_sample_document(collection):
    sample_doc = collection.find_one()
    if sample_doc:
        print("Sample document structure:")
        import json
        print(json.dumps(sample_doc, indent=2, default=str))
    else:
        print("No documents found in the collection.")


## print sample document
print_sample_document(collection)

Sample document structure:
{
  "_id": "67be09b7e3fd5d8914d8b14e",
  "plot": "Young Pauline is left a lot of money when her wealthy uncle dies. However, her uncle's secretary has been named as her guardian until she marries, at which time she will officially take ...",
  "runtime": 199.0,
  "genres": [
    "Action"
  ],
  "fullplot": "Young Pauline is left a lot of money when her wealthy uncle dies. However, her uncle's secretary has been named as her guardian until she marries, at which time she will officially take possession of her inheritance. Meanwhile, her \"guardian\" and his confederates constantly come up with schemes to get rid of Pauline so that he can get his hands on the money himself.",
  "directors": [
    "Louis J. Gasnier",
    "Donald MacKenzie"
  ],
  "writers": [
    "Charles W. Goddard (screenplay)",
    "Basil Dickey (screenplay)",
    "Charles W. Goddard (novel)",
    "George B. Seitz",
    "Bertram Millhauser"
  ],
  "countries": [
    "USA"
  ],
  "poster": "htt

## Setup Semantic Search functions

In [38]:
def semantic_search(user_query, collection):
    query_embedding = create_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Ensure query_embedding has 1024 dimensions
    if len(query_embedding) != 1024:
        print(f"Warning: Query embedding has {len(query_embedding)} dimensions, but index expects 1024")
        return "Embedding dimension mismatch"

    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",  # This matches your index name
                "queryVector": query_embedding,
                "path": "embedding",  # This is the correct field name as per your index
                "numCandidates": 150,
                "limit": 4,
            }
        },
        {
            "$project": {
                "_id": 0,
                "fullplot": 1,
                "title": 1,
                "genres": 1,
                "score": {"$meta": "vectorSearchScore"}
            }
        }
    ]

    try:
        results = list(collection.aggregate(pipeline))
        print(f"Number of results: {len(results)}")
        return results
    except Exception as e:
        print(f"Error in aggregation: {e}")
        return []

Note:
* Pydantic might be more useful for data validation of the results below.

In [39]:
## get search results function
def get_search_result(query, collection):
    """
    Function to get the search results. 
    """
    get_knowledge = semantic_search(query, collection) 

    search_result = ''
    ## format what is returned
    for result in get_knowledge:
        search_result += f"Title: {result.get('title', 'N/A')}, Plot: {result.get('fullplot', 'N/A')}\n"

    return search_result

## Test Query
* Test query using our embeddings WITHOUT an LLM. 

In [40]:
## query with retrieval of sources
query = "What is the best adventure movie to watch and why?"
source_information = get_search_result(query, collection)
combined_information = f"Query: {query}\nContinue to answer the query by using the Search Results:\n{source_information}."

print(combined_information)

Number of results: 4
Query: What is the best adventure movie to watch and why?
Continue to answer the query by using the Search Results:
Title: White Wolves: A Cry in the Wild II, Plot: A two-week trek through the Cascade Mountains tries the survival instincts of five adventurous teenagers. At first, it's all a good time. Shooting the rapids, exploring caves and making new friends. But when an accident occurs, Mother Nature raises the stakes and challenges the hikers to the greatest test of their young lives.
Title: Raiders of the Lost Ark, Plot: The year is 1936. An archeology professor named Indiana Jones is venturing in the jungles of South America searching for a golden statue. Unfortunately, he sets off a deadly trap but miraculously escapes. Then, Jones hears from a museum curator named Marcus Brody about a biblical artifact called The Ark of the Covenant, which can hold the key to humanly existence. Jones has to venture to vast places such as Nepal and Egypt to find this artifac

# Setup LLM -- Gemma via Hugging Face

In [41]:
from transformers import AutoTokenizer, AutoModelForCausalLM

## load tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")

## Im using a GPU so i will use below
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [42]:
## setup device agnostic code
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [44]:
# Define the chain of thought prompt
cot_prompt = """
Let's approach this step-by-step:

1. Analyze the query and identify the key elements we need to address.
2. Review the provided search results and extract relevant information.
3. Consider different adventure movies and their unique qualities.
4. Evaluate why certain adventure movies might be considered the best.
5. Formulate a comprehensive answer that addresses the query.

Now, let's answer the query:

"""

# Combine the original query, search results, and the chain of thought prompt
combined_information_with_cot = f"{combined_information}\n\n{cot_prompt}"

# Tokenize the input with the added chain of thought prompt
input_ids = tokenizer(combined_information_with_cot, return_tensors="pt").to(device)

# Generate the response
response = model.generate(**input_ids, max_new_tokens=500)

# Decode and print the response
print(tokenizer.decode(response[0]))

<bos>Query: What is the best adventure movie to watch and why?
Continue to answer the query by using the Search Results:
Title: White Wolves: A Cry in the Wild II, Plot: A two-week trek through the Cascade Mountains tries the survival instincts of five adventurous teenagers. At first, it's all a good time. Shooting the rapids, exploring caves and making new friends. But when an accident occurs, Mother Nature raises the stakes and challenges the hikers to the greatest test of their young lives.
Title: Raiders of the Lost Ark, Plot: The year is 1936. An archeology professor named Indiana Jones is venturing in the jungles of South America searching for a golden statue. Unfortunately, he sets off a deadly trap but miraculously escapes. Then, Jones hears from a museum curator named Marcus Brody about a biblical artifact called The Ark of the Covenant, which can hold the key to humanly existence. Jones has to venture to vast places such as Nepal and Egypt to find this artifact. However, he w

In [43]:
## this is a more simple approach to answering the query without cot prompting
## move tensors to GPU
input_ids = tokenizer(combined_information,
                     return_tensors="pt").to(device)

## get response --> unpack input and generate 500 new token response
response = model.generate(**input_ids, max_new_tokens=500)
## decode response
print(tokenizer.decode(response[0]))

<bos>Query: What is the best adventure movie to watch and why?
Continue to answer the query by using the Search Results:
Title: White Wolves: A Cry in the Wild II, Plot: A two-week trek through the Cascade Mountains tries the survival instincts of five adventurous teenagers. At first, it's all a good time. Shooting the rapids, exploring caves and making new friends. But when an accident occurs, Mother Nature raises the stakes and challenges the hikers to the greatest test of their young lives.
Title: Raiders of the Lost Ark, Plot: The year is 1936. An archeology professor named Indiana Jones is venturing in the jungles of South America searching for a golden statue. Unfortunately, he sets off a deadly trap but miraculously escapes. Then, Jones hears from a museum curator named Marcus Brody about a biblical artifact called The Ark of the Covenant, which can hold the key to humanly existence. Jones has to venture to vast places such as Nepal and Egypt to find this artifact. However, he w

# Summary
* We saw above the ins and outs of how to ingest data into MongoDB Atlast vector database and create vector indices.
* The end result was a simple RAG-LLM search and retrieval where the LLM Gemma found the most relevant documents to the query and recommended the best adventure movie to watch.
* There are other ways to enhance this and even evaluate the quality of the retrieval pipeline included but not limited to:

1. Various chunking strategies
2. Bi-encoder and Cross-encoders with Reranker
3. Hybrid search with reciprocal rank fusion (RRF) --> add BM25 or SPLADE algorithm for keyword search combined with vector search and the RRF algorithm to weight the keyword vs. semantic search results. 
4. Using different embedding models and testing their quality on the dataset and the retrieval system.
5. Pydantic for checking the data types and structure of the outputs.
6. Metadata filtering --> adding metadata to a query and the vector index can only enhance the retrieval process.
7. Evaluation --> there are various frameworks for evaluating a RAG system in the "Retrieval pipeline" and the "Generation pipeline" with associated metrics.
8. LLM fine-tuning --> we could certainly fine-tune the LLM on this dataset.
9. LLM prompt engineering --> we can add a prompt to the Gemma model to allow it to leverage in-context learning. I did experiment with CoT prompting above.
10. Agentic workflows --> adding agents can also help.
11. ....the list goes on...